PORCENTAJE DE HOGARES DONDE EL JEFE DE FAMILIA QUE RECIBE INGRESOS POR TRABAJO REALIZÓ LABORES DE CUIDADO SIN RECIBIR REMUNERACIÓN:

Descripción: Mide el porcentaje de hogares en la Ciudad de México que los jefes de hogar reciben ingresos por trabajo y que dedicaron tiempo a labores de cuidado sin recibir remuneración a cambio, respecto al total de hogares en la ciudad.

In [1]:
rm(list=ls())
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, readr, stringr, dplyr, survey)
#tiempo de espera
options(timeout = 300) 

Cargando paquete requerido: pacman



Se obtiene la tabla POBLACIÓN de la ENIGH 2022

In [2]:
#Descarga de archivos
url<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip"
##Creación de directorio temporal
td<- tempdir()
# Descarga del archivo temporal
tf = tempfile(tmpdir=td, fileext=".zip")
download.file(url, tf)
unzip(tf, files="poblacion.csv", exdir=td, 
      overwrite=TRUE)
fpath=file.path(td,"poblacion.csv")
unlink(td)
pob <- read_csv(fpath, show_col_types = FALSE)|>
  select(folioviv, foliohog, hor_4, entidad, numren, parentesco)|>
  filter(entidad =="09",
         parentesco == 101)


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


Se obtiene la tabla población CONCENTRADOHOGAR de la ENIGH 2022 y se unifica con POBLACIÓN

In [3]:
# URL del archivo ZIP para la tabla concentradohogar.csv
url2 <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_concentradohogar_csv.zip"

# Creación de directorio temporal
td2 <- tempdir()
# Descarga del archivo temporal
tf2 <- tempfile(tmpdir=td2, fileext=".zip")
download.file(url2, tf2)
unzip(tf2, files="concentradohogar.csv", exdir=td2, 
      overwrite=TRUE)
fpath2 <- file.path(td2, "concentradohogar.csv")
unlink(td2)


In [4]:
# Lectura del archivo CSV
concentrado <- read_csv(fpath2, show_col_types = FALSE)|>
  mutate(entidad = substr(folioviv,1,2))|>
  filter(entidad =="09")|>
  select(folioviv, foliohog, sexo_jefe, ingtrab, edad_jefe, upm, est_dis, factor)|>
  left_join(pob, by = c("folioviv", "foliohog"))|># Realizar la unión (left join) entre 'hogares' y 'vivienda' :)
  mutate(hog_cuidados = case_when(hor_4 != 0 | !is.na(hor_4) ~ 1, TRUE ~ 0), 
         hog_ing = case_when(ingtrab == 0 | is.na(ingtrab) ~ 0, TRUE ~ 1),
         hog_interestot = case_when(hog_ing == 1 & hog_cuidados == 1 ~ 1, TRUE ~ 0),
         hog_interesm = case_when(hog_ing == 1 & hog_cuidados == 1 & sexo_jefe == 2 ~ 1, TRUE ~ 0),
         hog_interesh = case_when(hog_ing == 1 & hog_cuidados == 1 & sexo_jefe == 1 ~ 1, TRUE ~ 0),
         )

In [5]:
View(concentrado)

folioviv,foliohog,sexo_jefe,ingtrab,edad_jefe,upm,est_dis,factor,hor_4,entidad,numren,parentesco,hog_cuidados,hog_ing,hog_interestot,hog_interesm,hog_interesh
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0900006901,1,2,34777.14,66,0003071,141,1578,28,09,01,101,1,1,1,1,0
0900006902,1,1,46222.78,71,0003071,141,1578,NA,09,01,101,0,1,0,0,0
0900006903,1,2,38396.73,40,0003071,141,1578,NA,09,01,101,0,1,0,0,0
0900006904,1,2,64687.47,50,0003071,141,1578,NA,09,01,101,0,1,0,0,0
0900006905,1,2,32038.03,83,0003071,141,1578,NA,09,01,101,0,1,0,0,0
0900006906,1,1,106300.27,40,0003071,141,1578,NA,09,01,101,0,1,0,0,0
0900009201,1,1,78464.05,54,0003072,141,1474,9,09,01,101,1,1,1,0,1
0900009202,1,1,172249.89,26,0003072,141,1474,NA,09,01,101,0,1,0,0,0
0900009204,1,2,35801.10,40,0003072,141,1474,NA,09,01,101,0,1,0,0,0


CÁLCULO DEL INDICADOR

In [6]:
##Diseño muestral
dmhog <- concentrado %>%
  as_survey_design(
    ids = upm,
    strata = est_dis,
    weights = factor,
    nest = TRUE
  )  
#total de hogares donde el jefe de familia se dedica tiempo de cuidados sin remuneración (total)
tot_hog <- dmhog %>%
  group_by(hog_interestot) %>%
  summarise(tot_hog = survey_total(vartype = "cv"),
            porcentaje_hogares = survey_prop(vartype = "cv")*100)
            
#total de hogares donde el jefe de familia se dedica tiempo de cuidados sin remuneración (mujeres)
tot_hogm <- dmhog %>%
  group_by(hog_interesm) %>%
  summarise(tot_hogm = survey_total(vartype = "cv"),
            porcentaje_hogares = survey_prop(vartype = "cv")*100)
#total de hogares donde el jefe de familia se dedica tiempo de cuidados sin remuneración (hombres)
tot_hogh <- dmhog %>%
  group_by(hog_interesh) %>%
  summarise(tot_hogh = survey_total(vartype = "cv"),
            porcentaje_hogares = survey_prop(vartype = "cv")*100)


In [11]:
show(tot_hog)

# A tibble: 2 × 5
  hog_interestot tot_hog tot_hog_cv porcentaje_hogares porcentaje_hogares_cv
           <dbl>   <dbl>      <dbl>              <dbl>                 <dbl>
1              0 2523080     0.0182               84.4                  1.11
2              1  466950     0.0611               15.6                  6.02


In [12]:
show(tot_hogh)

# A tibble: 2 × 5
  hog_interesh tot_hogh tot_hogh_cv porcentaje_hogares porcentaje_hogares_cv
         <dbl>    <dbl>       <dbl>              <dbl>                 <dbl>
1            0  2745582      0.0155              91.8                  0.746
2            1   244448      0.0853               8.18                 8.37 


In [14]:
show(tot_hogm)

# A tibble: 2 × 5
  hog_interesm tot_hogm tot_hogm_cv porcentaje_hogares porcentaje_hogares_cv
         <dbl>    <dbl>       <dbl>              <dbl>                 <dbl>
1            0  2767528      0.0160              92.6                  0.670
2            1   222502      0.0831               7.44                 8.34 
